In [1]:
import os
import numpy as np

import torch
from torchinfo import summary

# disable keras backend for transformers
os.environ["USE_TF"] = "0"
os.environ["USE_JAX"] = "0"

import evaluate
from datasets import load_dataset
from transformers import pipeline
from transformers import (
    AutoTokenizer,
    AutoModelForSequenceClassification,
    TrainingArguments,
    Trainer,
    DataCollatorWithPadding
)

In [2]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
device

device(type='cuda')

### Load the Dataset

In [3]:
dataset = load_dataset("imdb")
dataset

DatasetDict({
    train: Dataset({
        features: ['text', 'label'],
        num_rows: 25000
    })
    test: Dataset({
        features: ['text', 'label'],
        num_rows: 25000
    })
    unsupervised: Dataset({
        features: ['text', 'label'],
        num_rows: 50000
    })
})

In [4]:
dataset['train'][0]

{'text': 'I rented I AM CURIOUS-YELLOW from my video store because of all the controversy that surrounded it when it was first released in 1967. I also heard that at first it was seized by U.S. customs if it ever tried to enter this country, therefore being a fan of films considered "controversial" I really had to see this for myself.<br /><br />The plot is centered around a young Swedish drama student named Lena who wants to learn everything she can about life. In particular she wants to focus her attentions to making some sort of documentary on what the average Swede thought about certain political issues such as the Vietnam War and race issues in the United States. In between asking politicians and ordinary denizens of Stockholm about their opinions on politics, she has sex with her drama teacher, classmates, and married men.<br /><br />What kills me about I AM CURIOUS-YELLOW is that 40 years ago, this was considered pornographic. Really, the sex and nudity scenes are few and far be

### Load Base Model and Tokenizer

In [ ]:
base_model_name = "distilbert/distilbert-base-uncased"

tokenizer = AutoTokenizer.from_pretrained(base_model_name)
model = AutoModelForSequenceClassification.from_pretrained(base_model_name, num_labels=2).to(device)

def tokenize_function(examples):
    return tokenizer(
        examples["text"], 
        truncation=True
        )

tokenized_datasets = dataset.map(tokenize_function, batched=True)

# prepare smaller subset for quick fine-tuning
train_dataset = tokenized_datasets["train"].shuffle(seed=42).select(range(4000))
eval_dataset = tokenized_datasets["test"].shuffle(seed=42).select(range(1000))

summary(model, depth=5)

Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert/distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Map:   0%|          | 0/50000 [00:00<?, ? examples/s]

Layer (type:depth-idx)                                  Param #
DistilBertForSequenceClassification                     --
├─DistilBertModel: 1-1                                  --
│    └─Embeddings: 2-1                                  --
│    │    └─Embedding: 3-1                              23,440,896
│    │    └─Embedding: 3-2                              393,216
│    │    └─LayerNorm: 3-3                              1,536
│    │    └─Dropout: 3-4                                --
│    └─Transformer: 2-2                                 --
│    │    └─ModuleList: 3-5                             --
│    │    │    └─TransformerBlock: 4-1                  --
│    │    │    │    └─DistilBertSdpaAttention: 5-1      2,362,368
│    │    │    │    └─LayerNorm: 5-2                    1,536
│    │    │    │    └─FFN: 5-3                          4,722,432
│    │    │    │    └─LayerNorm: 5-4                    1,536
│    │    │    └─TransformerBlock: 4-2                  --
│    │    │    

### Train the Model

In [ ]:
accuracy = evaluate.load("accuracy")

def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    return accuracy.compute(predictions=predictions, references=labels)

training_args = TrainingArguments(
    output_dir="./distilbert_checkpoints",
    learning_rate=2e-5,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    num_train_epochs=2,
    weight_decay=0.01,
    eval_strategy="epoch",
    save_strategy="epoch",
    load_best_model_at_end=True,
    logging_dir="./logs",
    logging_steps=50,
    report_to="none",  # disables wandb/tensorboard by default
)

data_collator = DataCollatorWithPadding(
    tokenizer=tokenizer
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=eval_dataset,
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics,
)

# fine tune the model
trainer.train()

C:\Users\Nova\AppData\Local\Temp\ipykernel_6860\3266354839.py:26: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


Epoch,Training Loss,Validation Loss,Accuracy
1,0.377600,0.455203,0.824000
2,0.136700,0.352462,0.900000


TrainOutput(global_step=1000, training_loss=0.2814712381362915, metrics={'train_runtime': 1145.2306, 'train_samples_per_second': 6.985, 'train_steps_per_second': 0.873, 'total_flos': 1000253048039040.0, 'train_loss': 0.2814712381362915, 'epoch': 2.0})

### Evaluate the Model

In [18]:
metrics = trainer.evaluate()
for key, value in metrics.items():
    print(f"{key}: {value}")

eval_loss: 0.35246163606643677
eval_accuracy: 0.9
eval_runtime: 46.0541
eval_samples_per_second: 21.714
eval_steps_per_second: 2.714
epoch: 2.0


In [ ]:
# save the model
finetuned_model_path = "./distilbert-imdb-finetuned"
trainer.save_model(finetuned_model_path)
print(f"Fine-tuned model saved to: {finetuned_model_path}")

Fine-tuned model saved to: ./finetuned-distilbert-imdb


### Prediction

In [ ]:
finetuned_classifier = pipeline("sentiment-analysis", model=finetuned_model_path)

print(finetuned_classifier("The movie was surprisingly good and emotional!"))
print(finetuned_classifier("It was a total waste of time, I hated it."))

Device set to use cuda:0


[{'label': 'LABEL_1', 'score': 0.994044840335846}]
[{'label': 'LABEL_0', 'score': 0.9940481185913086}]
